In [3]:
import asyncio
from typing import List
from locast.live_candle.dydx.dydx_live_candle import DydxV4LiveCandle
from dydx_v4_client.network import TESTNET  # type: ignore
from locast.candle.dydx.dydx_resolution import DydxResolution


# This simulates a different component trying to query the live candle for data
async def price_query(live_candle: DydxV4LiveCandle, markets: List[str]) -> None:
    print("debug, entering price_query!")
    while live_candle.some_are_live():
        for market in markets:
            if live_candle.subscriptions.get(market) is True:
                if candle := live_candle.get_newest_ended_candle(market):
                    print(f"{market} CLOSE: {candle.close}")

        await asyncio.sleep(5)


async def unsubscribe_after_delay(
    live_candle: DydxV4LiveCandle,
    market: str,
    delay_sec: int,
) -> None:
    await asyncio.sleep(delay_sec)
    await live_candle.unsubscribe(market, DydxResolution.ONE_MINUTE.notation)


async def subscribe_after_delay(
    live_candle: DydxV4LiveCandle, market: str, delay_sec: int
) -> None:
    await asyncio.sleep(delay_sec)
    await live_candle.subscribe(market, DydxResolution.ONE_MINUTE.notation)


async def stop_after_delay(live_candle: DydxV4LiveCandle, delay_sec: int) -> None:
    await asyncio.sleep(delay_sec)
    await live_candle.stop()


async def main():
    live_candle = DydxV4LiveCandle(
        host_url=TESTNET.websocket_indexer,
        markets_per_resolution={
            DydxResolution.ONE_MINUTE.notation: ["ETH-USD", "BTC-USD"],
        },
    )

    await live_candle.start()
    tasks = [
        asyncio.create_task(price_query(live_candle, ["ETH-USD", "BTC-USD"])),
        asyncio.create_task(unsubscribe_after_delay(live_candle, "ETH-USD", 80)),
        asyncio.create_task(subscribe_after_delay(live_candle, "ETH-USD", 140)),
        asyncio.create_task(stop_after_delay(live_candle, 200)),
    ]
    await asyncio.gather(*tasks)


# asyncio.run(main()) # NOTE: Won't work in notebook
# Notebook solution:
loop = asyncio.get_event_loop()
loop.create_task(main())

<Task pending name='Task-5' coro=<main() running at /var/folders/57/whzhsmk962ndp1xbqwf8ypf00000gn/T/ipykernel_52876/3577153503.py:41>>

DEBUG:asyncio:Using selector: KqueueSelector


Starting live candle.


INFO:websocket:Websocket connected


Connection successful (id: d4ca1cc9-e4a5-4043-ad01-465672799c1c).Subscribing to 1MIN candles for ETH-USD.

Subscription successful (v4_candles, ETH-USD/1MIN).
Subscribing to 1MIN candles for BTC-USD.
Subscription successful (v4_candles, BTC-USD/1MIN).
debug, entering price_query!
Active BTC-USD candle started at: 2024-08-07 14:39:00+00:00
Active ETH-USD candle started at: 2024-08-07 14:39:00+00:00
Active ETH-USD candle started at: 2024-08-07 14:40:00+00:00
Newest ETH-USD candle started at: 2024-08-07 14:39:00+00:00

ETH-USD CLOSE: 2383.9
ETH-USD CLOSE: 2383.9
Active BTC-USD candle started at: 2024-08-07 14:40:00+00:00
Newest BTC-USD candle started at: 2024-08-07 14:39:00+00:00

Active ETH-USD candle started at: 2024-08-07 14:40:00+00:00
Newest ETH-USD candle started at: 2024-08-07 14:39:00+00:00

ETH-USD CLOSE: 2383.9
BTC-USD CLOSE: 55939
ETH-USD CLOSE: 2383.9
BTC-USD CLOSE: 55939
ETH-USD CLOSE: 2383.9
BTC-USD CLOSE: 55939
Unsubscribing from 1MIN candles for ETH-USD.
ETH-USD CLOSE: 238